In [3]:
import pretty_midi
import os

def load_midi_files(midi_dir):
    midi_notes = []
    for filename in os.listdir(midi_dir):
        if filename.endswith('.mid') or filename.endswith('.midi'):  # Check for MIDI files
            midi_path = os.path.join(midi_dir, filename)
            midi_data = pretty_midi.PrettyMIDI(midi_path)
            
            # Iterate over each instrument in the MIDI file
            for instrument in midi_data.instruments:
                if instrument.is_drum == False:  # Ignore drums
                    for note in instrument.notes:
                        midi_notes.append(note.pitch)  # Collect pitch of notes
    return midi_notes

# Example usage:
midi_dir = "D:\\archive"  # Path to your folder containing MIDI files
notes = load_midi_files(midi_dir)

# Print the first 10 notes to verify the result
print(notes[:10])


[71, 55, 59, 62, 71, 72, 67, 74, 72, 74]


In [4]:
import numpy as np
from tensorflow.keras.utils import to_categorical

# Prepare sequences of notes and their corresponding targets
def prepare_sequences(notes, seq_length=100):
    # Get unique pitch names (the notes that appear in your dataset)
    pitch_names = sorted(set(notes))
    
    # Create a mapping from pitch name to integer
    note_to_int = {note: number for number, note in enumerate(pitch_names)}
    int_to_note = {number: note for note, number in note_to_int.items()}

    # Prepare input sequences and output targets
    sequences = []
    targets = []
    for i in range(len(notes) - seq_length):
        seq_in = notes[i:i+seq_length]
        seq_out = notes[i+seq_length]
        sequences.append([note_to_int[note] for note in seq_in])
        targets.append(note_to_int[seq_out])
    
    # Reshape and normalize input sequences
    X = np.reshape(sequences, (len(sequences), seq_length, 1)) / float(len(pitch_names))
    
    # One-hot encode the output targets
    y = to_categorical(targets, num_classes=len(pitch_names))
    
    return X, y, int_to_note

# Example usage
seq_length = 100  # Length of each sequence
X, y, int_to_note = prepare_sequences(notes, seq_length)

# Check the shape of the input data
print(f"Input shape: {X.shape}")
print(f"Output shape: {y.shape}")


Input shape: (814932, 100, 1)
Output shape: (814932, 88)


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input

def build_model(seq_length, n_vocab):
    model = Sequential()
    
    # Use Input layer to define input shape
    model.add(Input(shape=(seq_length, 1)))
    
    # First LSTM layer
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))  # Dropout for regularization
    
    # Second LSTM layer
    model.add(LSTM(512))
    model.add(Dropout(0.3))  # Dropout for regularization
    
    # Dense layer
    model.add(Dense(256))
    
    # Output layer with softmax activation
    model.add(Dense(n_vocab, activation='softmax'))
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    
    return model

# Example usage
model = build_model(seq_length, len(int_to_note))
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 512)       │     1,052,672 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 512)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 512)            │     2,099,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 88)             │        22,616 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,816 (12.61 MB)

 Trainable params: 3,305,816 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# Train the model
def train_model(model, X, y, epochs=100, batch_size=32):
    model.fit(X, y, epochs=epochs, batch_size=batch_size,steps_per_epoch=50)

# Train for 50 epochs (adjust as needed)
train_model(model, X, y, epochs=10, batch_size=32)



Epoch 1/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 57s 1s/step - loss: 4.2513
Epoch 2/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 53s 1s/step - loss: 4.0385
Epoch 3/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - loss: 4.0794
Epoch 4/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 69s 1s/step - loss: 4.0378
Epoch 5/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - loss: 4.0174
Epoch 6/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 63s 1s/step - loss: 3.9968
Epoch 7/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 67s 1s/step - loss: 3.9502
Epoch 8/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 76s 2s/step - loss: 3.9961
Epoch 9/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 73s 1s/step - loss: 3.9257
Epoch 10/10
50/50 ━━━━━━━━━━━━━━━━━━━━ 72s 1s/step - loss: 3.9429


In [8]:
import random

# Generate music using the trained model
def generate_music(model, int_to_note, seq_length, n_notes=500):
    # Start with a random sequence from the training data
    start_idx = random.randint(0, len(X) - 1)
    pattern = X[start_idx]
    prediction_notes = []
    
    # Predict the next notes
    for _ in range(n_notes):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1)) / float(len(int_to_note))
        prediction = model.predict(prediction_input, verbose=0)
        index = np.argmax(prediction)
        result_note = int_to_note[index]
        prediction_notes.append(result_note)
        
        # Update the pattern with the predicted note
        pattern = np.append(pattern[1:], index)
    
    return prediction_notes

# Example usage to generate 500 notes
prediction_notes = generate_music(model, int_to_note, seq_length, n_notes=500)

# Print the generated notes (MIDI pitch values)
print(prediction_notes[:10])  # Print first 10 generated notes


[41, 41, 44, 44, 44, 44, 44, 44, 44, 44]


In [9]:
import pretty_midi

def create_midi(prediction_notes, output_file='D:/GeneratedMusic/generated_music1.mid'):
    # Check if directory exists and create it if not
    import os
    if not os.path.exists(os.path.dirname(output_file)):
        print("Directory does not exist. Creating...")
        os.makedirs(os.path.dirname(output_file))

    midi = pretty_midi.PrettyMIDI()

    # Select a program number (instrument). 0 corresponds to Acoustic Grand Piano
    instrument = pretty_midi.Instrument(program=0)  # 0 is for Acoustic Grand Piano

    start_time = 0
    for note in prediction_notes:
        midi_note = pretty_midi.Note(velocity=100, pitch=note, start=start_time, end=start_time + 1)
        instrument.notes.append(midi_note)
        start_time += 1

    midi.instruments.append(instrument)

    # Save the MIDI file
    print(f"Saving the MIDI file to: {output_file}")
    try:
        midi.write(output_file)
        print(f"Generated music saved as '{output_file}'")
    except Exception as e:
        print(f"Error saving the file: {e}")

# Example usage to save generated music to D drive
create_midi(prediction_notes, output_file='D:/GeneratedMusic/generated_music1.mid')




Saving the MIDI file to: D:/GeneratedMusic/generated_music1.mid
Generated music saved as 'D:/GeneratedMusic/generated_music1.mid'
